## 0. 필요한 라이브러리 및 모듈

In [1]:
# 판다스 넘파이
import pandas as pd
import numpy as np

# 머신러닝, 딥러닝 관련 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,accuracy_score, precision_score,roc_auc_score,f1_score,confusion_matrix
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression, LassoCV, Lasso
from sklearn.svm import SVC
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import label_binarize

import joblib

# 리샘플링 관련
from imblearn.over_sampling import BorderlineSMOTE, ADASYN
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

# 윈저라이징
from scipy.stats import mstats

# 성능평가
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler

# 통계
from scipy.stats import shapiro, levene, ttest_ind
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm


# 경고 메세지 숨기기
import warnings 
warnings.filterwarnings('ignore')

# 시각자료(그래프) 관련 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns

# 카운터함수
from collections import Counter

# 한글폰트 설정
# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 윈도우의 경우
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac의 경우
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 폰트 설정

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_14028\4121195870.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


##

In [3]:
# 파일 불러오기
df = pd.read_csv("data/코스피_코스닥_시장지표_결합.csv", index_col=0)
df.reset_index(drop=True)
df

,회사명,거래소코드,회계년도,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,...,장단기금리차(전월차)(%p),실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률
0,(주)CMG제약,58820,2012,109.147119,0.00,-341.571868,75.170505,-8.429134,-0.74,28.339513,...,0.11,3.1,316897731,2.89,1070.6,-2.367798,-0.724218,2.820,103.31,0.692008
1,(주)ES큐브,50120,2012,-16.575097,0.00,92.510776,1.564118,4.854248,-0.42,28.015517,...,0.11,3.1,316897731,2.89,1070.6,-2.367798,-0.724218,2.820,103.31,0.692008
2,(주)ES큐브,50120,2013,1.849357,0.00,-4481.062648,6.304012,4.692085,-57.63,24.759975,...,0.04,3.2,335647450,2.66,1055.4,-4.268391,-7.917155,2.858,101.66,-1.597135
3,(주)ES큐브,50120,2014,20.099174,0.00,96.260255,17.123515,-24.126991,-73.00,21.419545,...,-0.02,3.6,353600472,2.13,1099.3,-5.961831,-8.166371,2.098,101.12,-0.531182
4,(주)ES큐브,50120,2015,-1.751499,0.00,-274.898482,0.053133,-5.188115,44.44,27.803407,...,0.02,3.5,358513845,1.67,1172.5,-5.217313,-18.112831,1.662,97.06,-4.015032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12327,흥구석유(주),24060,2019,0.000000,-54.16,0.000000,0.000000,0.000000,-5.95,77.698333,...,0.12,3.5,397876077,1.53,1156.4,-3.353184,0.831050,1.360,100.46,0.029872
12328,흥구석유(주),24060,2020,0.000000,0.00,0.000000,0.000000,0.000000,-13.92,77.698333,...,0.03,4.2,430117234,0.66,1086.3,-5.204285,-9.500000,0.976,100.00,-0.457894
12329,흥구석유(주),24060,2021,0.000000,0.00,0.000000,0.000000,0.000000,4.41,77.698333,...,-0.02,3.4,438319248,1.29,1188.8,14.290000,14.987673,1.798,106.38,6.380000
12330,흥구석유(주),24060,2022,0.000000,0.00,0.000000,0.000000,0.000000,-2.82,77.698333,...,-0.35,3.0,399043064,3.98,1264.5,16.641876,20.573937,3.722,115.29,8.375635


In [3]:
# 전체 행, 전체 열을 보여주는 코드
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. 기초통계량

### 1.0 시각화

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

class Plot:
    
    def __init__(self, data):
        self.data = data
        self.num_plots = len(self.data.columns)
        self.num_cols = 6
        self.num_rows = (self.num_plots + self.num_cols - 1) // self.num_cols  # num_plots를 num_cols로 나눈 후 올림
    
    def make_sub_plot(self):
        # 서브플롯 생성     
        self.fig, self.axes = plt.subplots(self.num_rows, self.num_cols, figsize=(20, self.num_rows * 3))
    
    def del_sub_plot(self):
        # 빈 서브플롯 제거
        for j in range(self.num_plots, self.num_rows * self.num_cols):
            self.fig.delaxes(self.axes[j // self.num_cols, j % self.num_cols])

    def make_plot(self):
        # 서브플롯 간격 조정
        plt.subplots_adjust(hspace=0.4, wspace=0.4)  # 서브플롯 간의 간격 설정
        plt.tight_layout()
        plt.show()

    def m_boxplot(self):
        self.make_sub_plot()
        # 각 플롯에 해당 플롯 그리기
        
        for i, col in enumerate(self.data.columns):
            ax = self.axes[i // self.num_cols, i % self.num_cols]
            sns.boxplot(y=self.data[col], ax=ax)
            ax.set_title(col, rotation=0)  # 세로로 제목을 회전

        self.del_sub_plot()
        self.make_plot()

    def m_violinplot(self):
        self.make_sub_plot()
        # 각 플롯에 해당 플롯 그리기
        
        for i, col in enumerate(self.data.columns):
            ax = self.axes[i // self.num_cols, i % self.num_cols]
            sns.violinplot(y=self.data[col], ax=ax)
            ax.set_title(col, rotation=0)  # 세로로 제목을 회전

        self.del_sub_plot()
        self.make_plot()

    def m_histplot(self):
        self.make_sub_plot()
        # 각 플롯에 해당 플롯 그리기
        
        for i, col in enumerate(self.data.columns):
            ax = self.axes[i // self.num_cols, i % self.num_cols]
            sns.histplot(self.data[col], ax=ax)
            ax.set_title(col, rotation=0)  # 세로로 제목을 회전

        self.del_sub_plot()
        self.make_plot()
        
    def make_scatter(self):
        # scatter plot 그리기
        sns.pairplot(self.data)
        plt.show()


In [5]:
# selected_data = df.iloc[:, 2:-1]

# # Plot 클래스 인스턴스 생성 및 박스플롯 그리기
# p = Plot(selected_data)
# p.m_boxplot()
# p.m_violinplot()
# p.m_histplot()
# p.make_scatter()

### 1.1 전체 데이터 통계

In [5]:
df.describe()

,거래소코드,회계년도,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,매출액정상영업이익률,...,장단기금리차(전월차)(%p),실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률
count,12332.000000,12332.000000,12332.000000,12332.000000,1.233200e+04,12332.000000,12332.000000,12332.000000,12332.000000,12332.000000,...,12332.000000,12332.000000,1.233200e+04,12332.000000,12332.000000,12332.000000,12332.000000,12332.000000,12332.000000,12332.000000
mean,81903.846091,2017.354849,11.631108,44.913926,-2.617454e+02,15.261009,21.958582,4.128595,27.126661,-33.291456,...,-0.012176,3.455011,3.821329e+08,2.001956,1139.362561,1.452564,0.995568,2.089196,102.238735,1.477569
std,92114.074616,3.208399,81.066958,626.157766,2.766787e+04,168.459969,514.162800,53.032550,39.503810,3200.136640,...,0.146285,0.316356,3.632965e+07,0.915242,65.884103,8.095028,11.117174,0.783882,5.427765,3.631205
min,20.000000,2012.000000,-99.701756,-14737.300000,-3.043592e+06,-1229.876339,-99.707647,-99.710000,-3097.372431,-350051.850000,...,-0.350000,3.000000,3.168977e+08,0.660000,1055.400000,-5.961831,-18.112831,0.976000,95.300000,-4.015032
25%,14530.000000,2015.000000,0.000000,-15.620000,-2.984463e+01,0.000000,-1.524252,-3.130000,12.391830,2.140000,...,-0.020000,3.200000,3.585138e+08,1.520000,1070.600000,-5.204285,-8.166371,1.638000,100.000000,-0.531182
50%,52680.000000,2018.000000,2.799624,0.000000,0.000000e+00,3.794267,1.479038,1.020000,20.338039,5.390000,...,0.020000,3.500000,3.933325e+08,1.670000,1115.700000,-3.297016,-0.724218,1.817000,100.460000,0.029872
75%,107590.000000,2020.000000,12.118073,25.697500,4.894873e+01,12.238923,13.911903,7.442500,36.384292,10.502500,...,0.110000,3.600000,3.990431e+08,2.660000,1188.800000,6.142843,6.135630,2.820000,103.310000,3.441763
max,950070.000000,2022.000000,6181.288402,36408.400000,6.841035e+04,12465.900000,40693.187936,3478.130000,100.000000,82.100000,...,0.180000,4.200000,4.383192e+08,3.980000,1264.500000,16.641876,20.573937,3.722000,115.290000,8.375635


#### 1.1.1 box plot

#### 1.1.2 violin plot



### 1.2 정상 기업 통계

#### 1.2.1 정상기업 describe

In [6]:
# 정상기업 describe
df_0 = df[df['부실'] == 0]
df_0.describe()

,거래소코드,회계년도,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,매출액정상영업이익률,...,장단기금리차(전월차)(%p),실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률
count,11316.000000,11316.000000,11316.000000,11316.000000,11316.000000,11316.000000,11316.000000,11316.000000,11316.000000,11316.00000,...,11316.000000,11316.000000,1.131600e+04,11316.000000,11316.000000,11316.000000,11316.000000,11316.000000,11316.000000,11316.000000
mean,80951.593319,2017.470573,11.728886,49.716182,9.167239,14.108039,21.389198,4.602245,27.898703,6.92988,...,-0.014416,3.460834,3.834550e+08,1.986108,1141.028499,1.569532,1.091251,2.076097,102.237375,1.511631
std,92072.716740,3.156700,81.884815,644.751786,3909.410022,110.286550,519.500962,53.499681,27.461812,55.40759,...,0.147318,0.316532,3.562654e+07,0.919447,65.920342,8.167532,11.229227,0.787182,5.500340,3.670288
min,20.000000,2012.000000,-99.701756,-14737.300000,-339519.403249,-636.183557,-99.686397,-99.710000,-308.965727,-5737.15000,...,-0.350000,3.000000,3.168977e+08,0.660000,1055.400000,-5.961831,-18.112831,0.976000,95.300000,-4.015032
25%,14130.000000,2015.000000,0.000000,-17.930000,-27.948140,0.000000,-0.722538,-2.630000,12.878948,2.87000,...,-0.020000,3.200000,3.585138e+08,1.520000,1086.300000,-5.204285,-8.166371,1.638000,98.580000,-0.531182
50%,51910.000000,2018.000000,3.223661,0.000000,0.000000,4.322448,2.054561,1.280000,20.663572,5.94000,...,0.020000,3.500000,3.933325e+08,1.670000,1115.700000,-3.297016,0.831050,1.817000,100.460000,0.029872
75%,104830.000000,2020.000000,12.312381,29.612500,50.391887,12.460922,14.141636,7.580000,36.750730,11.08000,...,0.110000,3.600000,3.990431e+08,2.660000,1188.800000,6.142843,6.135630,2.820000,103.310000,3.441763
max,950070.000000,2022.000000,6181.288402,36408.400000,68410.347509,8527.244514,40693.187936,3478.130000,100.000000,82.10000,...,0.180000,4.200000,4.383192e+08,3.980000,1264.500000,16.641876,20.573937,3.722000,115.290000,8.375635


#### 1.2.2 정상기업 box plot

### 1.3 부실 기업 통계

In [7]:
# 정상기업 describe
df_0 = df[df['부실'] == 1]
df_0.describe()

,거래소코드,회계년도,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,매출액정상영업이익률,...,장단기금리차(전월차)(%p),실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률
count,1016.000000,1016.000000,1016.000000,1016.000000,1.016000e+03,1016.000000,1016.000000,1016.000000,1016.000000,1016.000000,...,1016.000000,1016.000000,1.016000e+03,1016.000000,1016.000000,1016.000000,1016.000000,1016.000000,1016.000000,1016.000000
mean,92509.842520,2016.065945,10.542076,-8.572618,-3.279115e+03,28.102561,28.300265,-1.146831,18.527809,-481.268455,...,0.012776,3.390157,3.674081e+08,2.178465,1120.807677,0.149799,-0.070120,2.235094,102.253888,1.098192
std,91954.194547,3.487145,71.353713,354.827099,9.549716e+04,457.159980,450.630675,47.223924,102.327911,11142.749779,...,0.131760,0.307159,4.060327e+07,0.847649,62.579397,7.113254,9.724223,0.730785,4.544001,3.140139
min,40.000000,2012.000000,-99.327840,-8363.750000,-3.043592e+06,-1229.876339,-99.707647,-98.940000,-3097.372431,-350051.850000,...,-0.350000,3.000000,3.168977e+08,0.660000,1055.400000,-5.961831,-18.112831,0.976000,95.300000,-4.015032
25%,24872.500000,2012.000000,-10.162086,0.000000,-9.484111e+01,-15.529300,-13.644571,-13.297500,6.673184,-26.415000,...,-0.020000,3.100000,3.168977e+08,1.530000,1070.600000,-4.268391,-7.917155,1.662000,100.000000,-0.531182
50%,62160.000000,2016.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,-1.875000,15.068449,-7.355000,...,0.030000,3.400000,3.617014e+08,2.130000,1099.300000,-2.367798,-0.724218,2.098000,101.120000,0.692008
75%,122927.500000,2019.000000,7.635420,0.000000,3.342644e+01,4.623762,9.462449,4.682500,31.233484,-0.017500,...,0.110000,3.500000,3.978761e+08,2.890000,1172.500000,0.747646,5.893729,2.820000,103.310000,1.876648
max,396270.000000,2022.000000,834.037855,2669.910000,6.463126e+03,12465.900000,10907.900318,1154.550000,100.000000,30.650000,...,0.180000,4.200000,4.383192e+08,3.980000,1264.500000,16.641876,20.573937,3.722000,115.290000,8.375635


### 1.4 정상 부실 비교

##

In [8]:
#  데이터프레임의 일부만 보여주는 코드
pd.options.display.max_rows = 60
pd.options.display.max_columns = 20


## 2. Train/Test Split

### 2.1 Train/Test Split

In [9]:
# X_data와 y_data 분리
X_data = df.drop(columns=['회사명', '거래소코드', '회계년도', '부실'])
y_data = df['부실']

In [10]:
# Train/Test Split
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data,test_size=0.3,stratify=y_data,random_state = 42)

#### 2.1.1 Train set과 Test set 차이

### 2.2 Train 통계량

In [11]:
X_train.describe()

,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,매출액정상영업이익률,매출액순이익률,자기자본정상영업이익률,...,장단기금리차(전월차)(%p),실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률
count,8632.000000,8632.000000,8632.000000,8632.000000,8632.000000,8632.000000,8632.000000,8632.000000,8632.000000,8632.000000,...,8632.000000,8632.000000,8.632000e+03,8632.000000,8632.000000,8632.000000,8632.000000,8632.000000,8632.000000,8632.000000
mean,12.097034,47.982512,-23.214590,15.864184,26.166153,4.204474,26.806215,-6.397771,-11.241434,8.287755,...,-0.011441,3.455155,3.821756e+08,1.997732,1139.449050,1.457356,0.945133,2.087541,102.232684,1.468127
std,88.920713,652.871750,4479.287644,171.901585,611.545122,47.028182,43.534964,651.478733,985.737201,18.199637,...,0.145633,0.316761,3.650379e+07,0.914708,65.872833,8.113683,11.156385,0.782847,5.420491,3.642691
min,-99.327840,-4532.630000,-339519.403249,-1229.876339,-99.707647,-99.640000,-3097.372431,-59379.020000,-90729.450000,-658.510000,...,-0.350000,3.000000,3.168977e+08,0.660000,1055.400000,-5.961831,-18.112831,0.976000,95.300000,-4.015032
25%,0.000000,-15.862500,-29.866310,0.000000,-1.594238,-3.280000,12.507308,2.160000,0.880000,3.200000,...,-0.020000,3.200000,3.585138e+08,1.520000,1070.600000,-5.204285,-8.166371,1.638000,100.000000,-0.531182
50%,2.882538,0.000000,0.000000,3.928972,1.351193,1.010000,20.211031,5.400000,4.230000,8.385000,...,0.020000,3.500000,3.933325e+08,1.670000,1115.700000,-3.297016,-0.724218,1.817000,100.460000,0.029872
75%,11.994430,25.100000,50.510303,12.340780,13.850962,7.500000,36.031574,10.430000,9.020000,14.890000,...,0.110000,3.600000,3.990431e+08,2.660000,1188.800000,6.142843,6.135630,2.820000,103.310000,3.441763
max,6181.288402,36408.400000,68410.347509,12465.900000,40693.187936,2414.830000,100.000000,67.070000,4378.760000,169.210000,...,0.180000,4.200000,4.383192e+08,3.980000,1264.500000,16.641876,20.573937,3.722000,115.290000,8.375635


#### 2.3.1 부실기업과 정상기업의 describe

#### 2.3.2. 부실기업과 정상기업의 히스토그램

## 3. winsorizing

In [12]:
class Win:
    def __init__(self):
        pass
    
    def winsorize_series(self, series, lower_bound, upper_bound):
        return np.clip(series, lower_bound, upper_bound)

    def apply_winsorizing(self, train_df, lower_quantile=0.01, upper_quantile=0.99):
        train_winsorized = train_df.copy()
        # test_winsorized = test_df.copy()
        
        for col in train_df.columns: 
            lower_bound = train_df[col].quantile(lower_quantile)
            upper_bound = train_df[col].quantile(upper_quantile)
            
            train_winsorized[col] = self.winsorize_series(train_df[col], lower_bound, upper_bound)
            # test_winsorized[col] = self.winsorize_series(test_df[col], lower_bound, upper_bound)
        
        return train_winsorized


In [13]:
# Win 클래스 인스턴스 생성 및 윈저라이징 적용
win = Win()
X_train = win.apply_winsorizing(X_train)

## 4. Scaling

In [14]:
class Scaling:
    def __init__(self, Train_data, Test_data):
        self.Train_data = Train_data
        self.Test_data = Test_data

    def Rbs(self):
        Rbscaler = RobustScaler()
        Rbscaler.fit(self.Train_data)
        X_scaled_train = Rbscaler.transform(self.Train_data)
        X_scaled_test = Rbscaler.transform(self.Test_data)
        
        train_scaled_df = pd.DataFrame(X_scaled_train, columns=self.Train_data.columns, index=self.Train_data.index)
        test_scaled_df = pd.DataFrame(X_scaled_test, columns=self.Test_data.columns, index=self.Test_data.index)
        
        return train_scaled_df, test_scaled_df

    def Stds(self):
        Stdscaler = StandardScaler()
        Stdscaler.fit(self.Train_data)
        X_scaled_train = Stdscaler.transform(self.Train_data)
        X_scaled_test = Stdscaler.transform(self.Test_data)
        
        train_scaled_df = pd.DataFrame(X_scaled_train, columns=self.Train_data.columns, index=self.Train_data.index)
        test_scaled_df = pd.DataFrame(X_scaled_test, columns=self.Test_data.columns, index=self.Test_data.index)
        
        return train_scaled_df, test_scaled_df

    def Mms(self):
        Mmscaler = MinMaxScaler()
        Mmscaler.fit(self.Train_data)
        X_scaled_train = Mmscaler.transform(self.Train_data)
        X_scaled_test = Mmscaler.transform(self.Test_data)
        
        train_scaled_df = pd.DataFrame(X_scaled_train, columns=self.Train_data.columns, index=self.Train_data.index)
        test_scaled_df = pd.DataFrame(X_scaled_test, columns=self.Test_data.columns, index=self.Test_data.index)
        
        return train_scaled_df, test_scaled_df


In [15]:
# Scaling 클래스 인스턴스 생성 및 스케일링 적용
scaling = Scaling(X_train, X_test)
X_train, X_test = scaling.Rbs()

## 5. Feature Selection

### 6.1 라쏘

In [16]:
import pandas as pd
from sklearn.linear_model import LassoCV

def get_feature_coefficients(X_train, y_train):
    # 라쏘 모델 초기화
    lasso = Lasso(alpha=0.001, max_iter=10000)
    lasso.fit(X_train, y_train)
    
    # 피처별 계수(coefficient)와 피처 이름 저장
    feature_coefficients = pd.DataFrame({'Feature': X_train.columns, 'Coefficient': lasso.coef_})
    # 계수의 절댓값을 기준으로 내림차순 정렬
    feature_coefficients = feature_coefficients.reindex(feature_coefficients['Coefficient'].abs().sort_values(ascending=False).index)
    # 회귀 계수가 0이 아닌 피처들만 선택
    feature_coefficients = feature_coefficients[feature_coefficients['Coefficient'] != 0]
    
    return feature_coefficients

# 함수 호출하여 결과 출력
feature_coefficients = get_feature_coefficients(X_train, y_train)
feature_coefficients.reset_index(drop=True)


,Feature,Coefficient
0,CASH FLOW 대 매출액비율,-0.053873
1,자기자본정상영업이익률,-0.049462
2,순운전자본회전률,-0.049094
3,사내유보 대 자기자본비율,-0.043720
4,CASH FLOW 대 부채비율,0.040516
5,CASH FLOW 대 총자본비율,-0.029438
6,금융비용부담률,0.029411
7,외환보유액(천달러),-0.022284
8,경기동행지수,-0.014253
9,타인자본회전률,-0.013198


In [17]:
list(feature_coefficients['Feature'])

['CASH FLOW 대 매출액비율',
 '자기자본정상영업이익률',
 '순운전자본회전률',
 '사내유보 대 자기자본비율',
 'CASH FLOW 대 부채비율',
 'CASH FLOW 대 총자본비율',
 '금융비용부담률',
 '외환보유액(천달러)',
 '경기동행지수',
 '타인자본회전률',
 '유동비율',
 '매출채권회전률',
 '매출액순이익률',
 'GDP대비국가부채비율',
 '유동자산회전률',
 '총자산순이익률',
 '자기자본회전률',
 '생산자물가지수',
 '총자산영업이익률',
 '생산자물가지수증감률',
 '당좌비율',
 'CP전년대비증가율',
 '부가가치율',
 '수지비율(관계기업투자손익 제외)',
 '매출액증가율',
 'CPI',
 '실질GDP성장률',
 '1주당매출액',
 '1주당순자산',
 '총자본투자효율',
 '현금비율',
 '노동소득분배율',
 '1주당 CASH FLOW',
 '종업원수증가율',
 '매출액총이익률',
 '매출액정상영업이익률',
 '정상영업이익증가율',
 '자본집약도',
 '설비투자효율',
 '1주당순이익',
 '매출채권 대 매입채무비율',
 '자기자본증가율',
 '운전자본회전률',
 '순이익증가율',
 'CASH FLOW 대 차입금비율',
 '기계장비율',
 '재고자산 대 순운전자본비율',
 '재고자산회전률',
 '종업원1인당 부가가치',
 '기계투자효율']

In [18]:
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, r2_score

# 라쏘 모델의 알파값 후보 리스트
alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

# 최적의 알파값 초기화
best_alpha = None
best_rmse = float('inf')  # 무한대로 초기화
best_r2 = -float('inf')    # 음의 무한대로 초기화

# 각 알파값에 대한 모델 훈련 및 평가
for alpha in alphas:
    lasso_model = LassoCV(alphas=[alpha], cv=5)
    lasso_model.fit(X_train, y_train)
    
    # 예측값 계산
    y_pred = lasso_model.predict(X_train)
    
    # RMSE 계산
    rmse = mean_squared_error(y_train, y_pred, squared=False)
    
    # R^2 계산
    r2 = r2_score(y_train, y_pred)
    
    # RMSE와 R^2가 기존 최적값보다 우수한 경우 최적값 업데이트
    if rmse < best_rmse:
        best_rmse = rmse
        best_alpha_rmse = alpha
    
    if r2 > best_r2:
        best_r2 = r2
        best_alpha_r2 = alpha

print("Best alpha based on RMSE:", best_alpha_rmse)
print("Best RMSE:", best_rmse)

print("Best alpha based on R^2:", best_alpha_r2)
print("Best R^2:", best_r2)


Best alpha based on RMSE: 0.001
Best RMSE: 0.20623059368610477
Best alpha based on R^2: 0.001
Best R^2: 0.4372967372729809


### 5.2 t-test

In [19]:
from scipy.stats import shapiro, levene, ttest_ind

# 정규성 확인 (Shapiro-Wilk 검정)
normality_results = {}
for column in X_train.columns:
    stat_group1, p_group1 = shapiro(X_train.loc[y_train == 0, column])
    stat_group2, p_group2 = shapiro(X_train.loc[y_train == 1, column])
    normality_results[column] = {'Group 1': (stat_group1, p_group1), 'Group 2': (stat_group2, p_group2)}

print('Shapiro-Wilk test - Normality:')
for column, results in sorted(normality_results.items(), key=lambda x: x[1]['Group 1'][1]):
    stat_group1, p_group1 = results['Group 1']
    stat_group2, p_group2 = results['Group 2']
    print('%s: Group 1 - stat=%.3f, p-value=%.3f; Group 2 - stat=%.3f, p-value=%.3f' % (column, stat_group1, p_group1, stat_group2, p_group2))

# 등분산성 확인 (Levene's test)
variance_results = {}
for column in X_train.columns:
    stat_levene, p_levene = levene(X_train.loc[y_train == 0, column], X_train.loc[y_train == 1, column])
    variance_results[column] = (stat_levene, p_levene)

print('Levene\'s test for Homogeneity of Variance:')
for column, (stat_levene, p_levene) in variance_results.items():
    print('%s: stat=%.3f, p-value=%.3f' % (column, stat_levene, p_levene))

# t-검정 수행
t_test_results = {}
for column in X_train.columns:
    equal_var = variance_results[column][1] > 0.05  # 등분산성 가정이 성립하는 경우
    t_stat, p_ttest = ttest_ind(X_train.loc[y_train == 0, column], X_train.loc[y_train == 1, column], equal_var=equal_var)
    t_test_results[column] = (t_stat, p_ttest)

print('t-test results:')
for column, (t_stat, p_ttest) in sorted(t_test_results.items(), key=lambda x: x[1]):
    print('%s: stat=%.3f, p-value=%f' % (column, t_stat, p_ttest))

# p-값이 작은 순서대로 변수명 추출
sorted_columns_by_p_value = [column for column, (_, p_value) in sorted(t_test_results.items(), key=lambda x: x[1]) if p_value < 0.05][::-1]


Shapiro-Wilk test - Normality:
기계투자효율: Group 1 - stat=0.116, p-value=0.000; Group 2 - stat=0.098, p-value=0.000
재고자산회전률: Group 1 - stat=0.203, p-value=0.000; Group 2 - stat=0.174, p-value=0.000
CASH FLOW 대 차입금비율: Group 1 - stat=0.265, p-value=0.000; Group 2 - stat=0.205, p-value=0.000
자본집약도: Group 1 - stat=0.267, p-value=0.000; Group 2 - stat=0.343, p-value=0.000
매출채권 대 매입채무비율: Group 1 - stat=0.321, p-value=0.000; Group 2 - stat=0.325, p-value=0.000
기계장비율: Group 1 - stat=0.331, p-value=0.000; Group 2 - stat=0.385, p-value=0.000
설비투자효율: Group 1 - stat=0.357, p-value=0.000; Group 2 - stat=0.363, p-value=0.000
운전자본회전률: Group 1 - stat=0.368, p-value=0.000; Group 2 - stat=0.416, p-value=0.000
1주당순이익: Group 1 - stat=0.402, p-value=0.000; Group 2 - stat=0.233, p-value=0.000
1주당 CASH FLOW: Group 1 - stat=0.435, p-value=0.000; Group 2 - stat=0.269, p-value=0.000
1주당매출액: Group 1 - stat=0.439, p-value=0.000; Group 2 - stat=0.231, p-value=0.000
1주당순자산: Group 1 - stat=0.459, p-value=0.000; Group 2 

In [20]:
import pandas as pd

# p-value가 0.05보다 작은 경우만 선택하고, 해당 변수명과 p-value를 DataFrame에 저장
significant_results = []
for column, (_, p_value) in sorted(t_test_results.items(), key=lambda x: x[1]):
    if p_value < 0.05:
        significant_results.append((column, p_value))

# 데이터프레임으로 변환
significant_df = pd.DataFrame(significant_results, columns=['Variable', 'p-value'])

# p-value가 작은 순서대로 정렬
significant_df = significant_df.sort_values(by='p-value')

significant_df.reset_index(drop=True)


,Variable,p-value
0,CASH FLOW 대 부채비율,4.383165e-238
1,CASH FLOW 대 총자본비율,4.027445e-179
2,총자산영업이익률,1.920238e-176
3,타인자본회전률,4.793997e-165
4,자기자본정상영업이익률,3.540314e-159
5,유보율,2.736520e-140
6,총자산순이익률,4.211436e-135
7,총자본투자효율,1.279818e-124
8,순운전자본회전률,3.422969e-115
9,자기자본순이익률,2.873526e-113


In [21]:
sorted_columns_by_p_value

['CASH FLOW 대 부채비율',
 'CASH FLOW 대 총자본비율',
 '총자산영업이익률',
 '자기자본정상영업이익률',
 '타인자본회전률',
 '총자산순이익률',
 '총자본투자효율',
 '유보율',
 '자기자본순이익률',
 'CASH FLOW 대 매출액비율',
 '순운전자본회전률',
 '매출액정상영업이익률',
 '사내유보 대 자기자본비율',
 '1주당 CASH FLOW',
 '매출액순이익률',
 'CASH FLOW 대 차입금비율',
 '부가가치율',
 '자기자본구성비율',
 '종업원1인당 부가가치',
 '총자본회전률',
 '1주당매출액',
 '1주당순자산',
 '1주당순이익',
 '유동자산회전률',
 '경기동행지수',
 '외환보유액(천달러)',
 '경기선행지수',
 '당좌자산회전률',
 '순이익증가율',
 'GDP대비국가부채비율',
 '환율',
 '매출액총이익률',
 '실업률',
 '설비투자효율',
 '정상영업이익증가율',
 '종업원수증가율',
 '수출물가지수증감률',
 '유동비율',
 '자기자본증가율',
 '당좌비율',
 '현금비율',
 '매출액증가율',
 '매출채권회전률',
 '총자본증가율',
 '생산자물가지수증감률',
 '수입물가지수증감률',
 'CP전년대비증가율',
 '노동장비율',
 '장단기금리차(전월차)(%p)',
 '3년국고채수익률',
 'CD이자율',
 '부채비율',
 '금융비용부담률',
 '수지비율(관계기업투자손익 제외)']

In [22]:
intersection_features = [feature for feature in sorted_columns_by_p_value if feature in set(feature_coefficients['Feature'])]

In [23]:
len(intersection_features)

38

In [24]:
intersection_features

['CASH FLOW 대 부채비율',
 'CASH FLOW 대 총자본비율',
 '총자산영업이익률',
 '자기자본정상영업이익률',
 '타인자본회전률',
 '총자산순이익률',
 '총자본투자효율',
 'CASH FLOW 대 매출액비율',
 '순운전자본회전률',
 '매출액정상영업이익률',
 '사내유보 대 자기자본비율',
 '1주당 CASH FLOW',
 '매출액순이익률',
 'CASH FLOW 대 차입금비율',
 '부가가치율',
 '종업원1인당 부가가치',
 '1주당매출액',
 '1주당순자산',
 '1주당순이익',
 '유동자산회전률',
 '경기동행지수',
 '외환보유액(천달러)',
 '순이익증가율',
 'GDP대비국가부채비율',
 '매출액총이익률',
 '설비투자효율',
 '정상영업이익증가율',
 '종업원수증가율',
 '유동비율',
 '자기자본증가율',
 '당좌비율',
 '현금비율',
 '매출액증가율',
 '매출채권회전률',
 '생산자물가지수증감률',
 'CP전년대비증가율',
 '금융비용부담률',
 '수지비율(관계기업투자손익 제외)']

In [52]:
X_train_two = X_train[['CASH FLOW 대 부채비율','CASH FLOW 대 총자본비율','자기자본정상영업이익률',
                        '타인자본회전률', '총자본투자효율','순운전자본회전률', '부가가치율', '경기동행지수',
                        '외환보유액(천달러)', '순이익증가율','정상영업이익증가율','당좌비율',
                        ]]

## 유동비율, 총자산영업이익률 제거(VIF수치 확인 후 높은 것)

In [53]:
def calculate_vif(X):
    # X = add_constant(X)
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

vif_result = calculate_vif(X_train_two)
vif_result.sort_values(by='VIF', ascending=False).reset_index(drop=True)

,Feature,VIF
0,외환보유액(천달러),8.501342
1,경기동행지수,8.295037
2,총자본투자효율,5.717154
3,CASH FLOW 대 총자본비율,5.683577
4,CASH FLOW 대 부채비율,4.609189
5,부가가치율,4.132913
6,타인자본회전률,3.198857
7,자기자본정상영업이익률,3.043708
8,순운전자본회전률,2.971946
9,당좌비율,1.751627


## 6. Resampling

In [54]:
class Resampling:
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    def bds(self):
        X_resampled_train, y_resampled_train = BorderlineSMOTE(random_state=0).fit_resample(self.X_train, self.y_train)    
        return X_resampled_train, y_resampled_train

    def tomek(self):
        smoteto = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
        X_resampled_train, y_resampled_train = smoteto.fit_resample(self.X_train, self.y_train)    
        return X_resampled_train, y_resampled_train

    def adasyn(self):
        X_resampled_train, y_resampled_train = ADASYN(random_state=42).fit_resample(self.X_train, self.y_train)    
        return X_resampled_train, y_resampled_train    


In [55]:
rs = Resampling(X_train, y_train)
X_train, y_train = rs.tomek()

In [56]:
X_train_two = X_train[['CASH FLOW 대 부채비율','CASH FLOW 대 총자본비율','자기자본정상영업이익률',
                        '타인자본회전률', '총자본투자효율','순운전자본회전률', '부가가치율', '경기동행지수',
                        '외환보유액(천달러)', '순이익증가율','정상영업이익증가율','당좌비율',
                        ]]
X_test_two = X_test[['CASH FLOW 대 부채비율','CASH FLOW 대 총자본비율','자기자본정상영업이익률',
                        '타인자본회전률', '총자본투자효율','순운전자본회전률', '부가가치율', '경기동행지수',
                        '외환보유액(천달러)', '순이익증가율','정상영업이익증가율','당좌비율',
                        ]]

## 7. Modeling(Hyperparameter Tuning) 

In [34]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [35]:
import xgboost as xgb
print(xgb.__version__)

2.0.3


In [58]:
import joblib
import numpy as np
import pandas as pd
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import label_binarize

class Modeling:
    def __init__(self, X_train, X_test, y_train, y_test):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test

        self.model_params = {
            'LogisticRegression': (LogisticRegression(solver='liblinear'), {'C': [0.1, 1, 10, 43], 'penalty': ['l1', 'l2']}),
            'DecisionTree': (DecisionTreeClassifier(), {'max_depth': [4, 10, 20], 'min_samples_leaf': [6, 10, 3]}),
            'RandomForest': (RandomForestClassifier(), {'n_estimators': [100, 200, 300, 10, 20], 'max_depth': [None, 10, 20, 4, 5]}),
            'GradientBoosting': (GradientBoostingClassifier(), {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2]}),
            'XGBoost': (XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {'learning_rate': [0.01, 0.1, 0.2, 0.03]}),
            'LightGBM': (LGBMClassifier(), {'learning_rate': [0.01, 0.1, 0.2, 0.03]}),
            'CatBoost': (CatBoostClassifier(silent=True), {'iterations': [100, 200, 300], 'learning_rate': [0.01, 0.1, 0.2]})
        }

    def fit_and_evaluate(self, model_name):
        model, params = self.model_params[model_name]
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        grid_search = GridSearchCV(model, params, cv=kfold, scoring='recall', n_jobs=-1)

        if model_name in ['XGBoost', 'LightGBM', 'CatBoost']:
            eval_set = [(self.X_train, self.y_train), (self.X_test, self.y_test)]
            cloned_model = clone(model)
            grid_search = GridSearchCV(cloned_model, params, cv=kfold, scoring='recall', n_jobs=-1)
            grid_search.fit(self.X_train, self.y_train, eval_set=eval_set, early_stopping_rounds=10)
        else:
            grid_search.fit(self.X_train, self.y_train)

        best_model = grid_search.best_estimator_
        y_pred_train = best_model.predict(self.X_train)
        y_pred_test = best_model.predict(self.X_test)

        y_train_bin = label_binarize(self.y_train, classes=np.unique(self.y_train))
        y_test_bin = label_binarize(self.y_test, classes=np.unique(self.y_test))

        if y_train_bin.shape[1] == 1:
            y_train_bin = np.hstack([1 - y_train_bin, y_train_bin])
        if y_test_bin.shape[1] == 1:
            y_test_bin = np.hstack([1 - y_test_bin, y_test_bin])

        metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

        train_scores = [
            accuracy_score(self.y_train, y_pred_train),
            precision_score(self.y_train, y_pred_train, average='weighted'),
            recall_score(self.y_train, y_pred_train, average='weighted'),
            f1_score(self.y_train, y_pred_train, average='weighted'),
            roc_auc_score(y_train_bin, best_model.predict_proba(self.X_train), average='weighted')
        ]

        test_scores = [
            accuracy_score(self.y_test, y_pred_test),
            precision_score(self.y_test, y_pred_test, average='weighted'),
            recall_score(self.y_test, y_pred_test, average='weighted'),
            f1_score(self.y_test, y_pred_test, average='weighted'),
            roc_auc_score(y_test_bin, best_model.predict_proba(self.X_test), average='weighted')
        ]

        result_df = pd.DataFrame({
            'Metric': metrics,
            f'{model_name}_train': train_scores,
            f'{model_name}_test': test_scores
        }).set_index('Metric')

        joblib_file = f"./RBS/best_{model_name}_model.pkl"
        joblib.dump(best_model, joblib_file)
        print(f"Best model saved as {joblib_file}")

        print(f'{model_name}:')
        print(f'  최적의 하이퍼파라미터: {grid_search.best_params_}')
        print(f'  K-fold 교차 검증 재현율: {grid_search.best_score_:.4f}')

        return result_df


In [36]:
# import joblib
# import numpy as np
# import pandas as pd
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# from sklearn.model_selection import GridSearchCV, KFold
# from sklearn.preprocessing import label_binarize
# from sklearn.base import clone

# class Modeling:
#     def __init__(self, X_train, X_test, y_train, y_test):
#         self.X_train = X_train
#         self.X_test = X_test
#         self.y_train = y_train
#         self.y_test = y_test

#         self.model_params = {
#             'LogisticRegression': (LogisticRegression(solver='liblinear'), {'C': [0.1, 1, 10, 43], 'penalty': ['l1', 'l2']}),
#             'DecisionTree': (DecisionTreeClassifier(), {'max_depth': [4, 10, 20], 'min_samples_leaf': [6, 10, 3]}),
#             'RandomForest': (RandomForestClassifier(), {'n_estimators': [100, 200, 300, 10, 20], 'max_depth': [None, 10, 20, 4, 5]}),
#             'GradientBoosting': (GradientBoostingClassifier(), {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2]}),
#             'XGBoost': (XGBClassifier(use_label_encoder=False, eval_metric='logloss', early_stopping_rounds=10, n_estimators=1000), {'learning_rate': [0.01, 0.1, 0.2, 0.03]}),
#             'LightGBM': (LGBMClassifier(early_stopping_round=10, n_estimators=1000), {'learning_rate': [0.01, 0.1, 0.2, 0.03]}),
#             'CatBoost': (CatBoostClassifier(silent=True, early_stopping_rounds=10), {'iterations': [100, 200, 300], 'learning_rate': [0.01, 0.1, 0.2]})
#         }

#     def fit_and_evaluate(self, model_name):
#         model, params = self.model_params[model_name]
#         kfold = KFold(n_splits=5, shuffle=True, random_state=42)
#         grid_search = GridSearchCV(model, params, cv=kfold, scoring='recall', n_jobs=-1)

#         eval_set = None
#         if model_name in ['XGBoost', 'LightGBM', 'CatBoost']:
#             eval_set = [(self.X_train, self.y_train), (self.X_test, self.y_test)]
#             cloned_model = clone(model)
#             grid_search = GridSearchCV(cloned_model, params, cv=kfold, scoring='recall', n_jobs=-1)
#             grid_search.fit(self.X_train, self.y_train, eval_set=eval_set)
#         else:
#             grid_search.fit(self.X_train, self.y_train)
        
#         best_model = grid_search.best_estimator_
#         y_pred_train = best_model.predict(self.X_train)
#         y_pred_test = best_model.predict(self.X_test)

#         y_train_bin = label_binarize(self.y_train, classes=np.unique(self.y_train))
#         y_test_bin = label_binarize(self.y_test, classes=np.unique(self.y_test))
        
#         if y_train_bin.shape[1] == 1:
#             y_train_bin = np.hstack([1 - y_train_bin, y_train_bin])
#         if y_test_bin.shape[1] == 1:
#             y_test_bin = np.hstack([1 - y_test_bin, y_test_bin])
        
#         metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
        
#         train_scores = [
#             accuracy_score(self.y_train, y_pred_train),
#             precision_score(self.y_train, y_pred_train, average='weighted'),
#             recall_score(self.y_train, y_pred_train, average='weighted'),
#             f1_score(self.y_train, y_pred_train, average='weighted'),
#             roc_auc_score(y_train_bin, best_model.predict_proba(self.X_train), average='weighted')
#         ]
        
#         test_scores = [
#             accuracy_score(self.y_test, y_pred_test),
#             precision_score(self.y_test, y_pred_test, average='weighted'),
#             recall_score(self.y_test, y_pred_test, average='weighted'),
#             f1_score(self.y_test, y_pred_test, average='weighted'),
#             roc_auc_score(y_test_bin, best_model.predict_proba(self.X_test), average='weighted')
#         ]

#         result_df = pd.DataFrame({
#             'Metric': metrics,
#             f'{model_name}_train': train_scores,
#             f'{model_name}_test': test_scores
#         }).set_index('Metric')
        
#         joblib_file = f"./RBS/best_{model_name}_model.pkl"
#         joblib.dump(best_model, joblib_file)
#         print(f"Best model saved as {joblib_file}")

#         print(f'{model_name}:')
#         print(f'  최적의 하이퍼파라미터: {grid_search.best_params_}')
#         print(f'  K-fold 교차 검증 정확도: {grid_search.best_score_:.4f}')
        
#         return result_df



In [59]:
# 전체 행, 전체 열을 보여주는 코드
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [60]:
# Example of usage
# Assuming you have already split your data into X_train, X_test, y_train, y_test

list_model = ['LogisticRegression', 'DecisionTree', 'RandomForest', 'GradientBoosting', 'XGBoost', 'LightGBM', 'CatBoost']
modeling_instance = Modeling(X_train, X_test, y_train, y_test)
df = pd.DataFrame()

for i in list_model:
    result = modeling_instance.fit_and_evaluate(i)
    df = pd.concat([df, result], axis=1)

df


Best model saved as ./RBS/best_LogisticRegression_model.pkl
LogisticRegression:
  최적의 하이퍼파라미터: {'C': 0.1, 'penalty': 'l2'}
  K-fold 교차 검증 재현율: 0.9314
Best model saved as ./RBS/best_DecisionTree_model.pkl
DecisionTree:
  최적의 하이퍼파라미터: {'max_depth': 10, 'min_samples_leaf': 3}
  K-fold 교차 검증 재현율: 0.9605
Best model saved as ./RBS/best_RandomForest_model.pkl
RandomForest:
  최적의 하이퍼파라미터: {'max_depth': 10, 'n_estimators': 100}
  K-fold 교차 검증 재현율: 0.9884
Best model saved as ./RBS/best_GradientBoosting_model.pkl
GradientBoosting:
  최적의 하이퍼파라미터: {'learning_rate': 0.2, 'n_estimators': 150}
  K-fold 교차 검증 재현율: 0.9793


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

## 한개씩 돌리는 파일

In [42]:
logit = LogisticRegression()
logit.fit(X_train, y_train)
y_pred_train = logit.predict(X_train)
y_pred_test = logit.predict(X_test)

log_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
log_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['logit_train'] = log_train
df['logit_test'] = log_test

print(confusion_matrix(y_test,y_pred_test))

[[3095  300]
 [  39  266]]


In [43]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)

rf_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
rf_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['rf_train'] = rf_train
df['rf_test'] = rf_test


print(confusion_matrix(y_test,y_pred_test))

[[3239  156]
 [  49  256]]


In [44]:
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)
y_pred_train = xgb.predict(X_train)
y_pred_test = xgb.predict(X_test)

xgb_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
xgb_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['xgb_train'] = xgb_train
df['xgb_test'] = xgb_test


print(confusion_matrix(y_test,y_pred_test))

[[3274  121]
 [  73  232]]


In [45]:
lgb = LGBMClassifier(random_state=42)
lgb.fit(X_train, y_train)
y_pred_train = lgb.predict(X_train)
y_pred_test = lgb.predict(X_test)

lgb_train =[accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
lgb_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['lgb_train'] = lgb_train
df['lgb_test'] = lgb_test


print(confusion_matrix(y_test,y_pred_test))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 7921, number of negative: 7917
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002002 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3570
[LightGBM] [Info] Number of data points in the train set: 15838, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500126 -> initscore=0.000505
[LightGBM] [Info] Start training from score 0.000505
[[3278  117]
 [  60  245]]


In [46]:
cat = CatBoostClassifier(random_state=42, verbose=0)
cat.fit(X_train, y_train)
y_pred_train = cat.predict(X_train)
y_pred_test = cat.predict(X_test)

cat_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
cat_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['cat_train'] = cat_train
df['cat_test'] = cat_test


print(confusion_matrix(y_test,y_pred_test))

[[3272  123]
 [  65  240]]


In [47]:
svc_clf = SVC() 
svc_clf.fit(X_train,y_train)
y_pred_train = svc_clf.predict(X_train)
y_pred_test = svc_clf.predict(X_test)

svc_clf_train =[accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
svc_clf_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['svm_train'] = svc_clf_train
df['svm_test'] = svc_clf_test


print(confusion_matrix(y_test,y_pred_test))

[[3151  244]
 [  84  221]]


In [289]:
clf = TabNetClassifier()

# 모델 훈련
clf.fit(
    X_train.values, y_train.values,
    eval_set=[(X_train.values, y_train.values), (X_test.values, y_test.values)],
    eval_name=['train', 'test'],
    eval_metric=['accuracy'],
    max_epochs=10,
    patience=10,
    batch_size=1024, 
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

epoch 0  | loss: 0.29479 | train_accuracy: 0.92359 | test_accuracy: 0.9267  |  0:00:02s
epoch 1  | loss: 0.17701 | train_accuracy: 0.93078 | test_accuracy: 0.93075 |  0:00:07s
epoch 2  | loss: 0.1571  | train_accuracy: 0.93252 | test_accuracy: 0.93319 |  0:00:10s
epoch 3  | loss: 0.15507 | train_accuracy: 0.93786 | test_accuracy: 0.93698 |  0:00:13s
epoch 4  | loss: 0.15467 | train_accuracy: 0.93519 | test_accuracy: 0.93698 |  0:00:16s
epoch 5  | loss: 0.14796 | train_accuracy: 0.93577 | test_accuracy: 0.93806 |  0:00:19s
epoch 6  | loss: 0.14526 | train_accuracy: 0.93739 | test_accuracy: 0.93914 |  0:00:22s
epoch 7  | loss: 0.14435 | train_accuracy: 0.93925 | test_accuracy: 0.94347 |  0:00:25s
epoch 8  | loss: 0.14055 | train_accuracy: 0.94122 | test_accuracy: 0.94374 |  0:00:28s
epoch 9  | loss: 0.14094 | train_accuracy: 0.94157 | test_accuracy: 0.94266 |  0:00:31s
Stop training because you reached max_epochs = 10 with best_epoch = 8 and best_test_accuracy = 0.94374


In [290]:
y_pred_train = clf.predict(X_train.values)
y_pred_test = clf.predict(X_test.values)

tab_train = [accuracy_score(y_train, y_pred_train),precision_score(y_train, y_pred_train),recall_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train)]
tab_test = [accuracy_score(y_test, y_pred_test),precision_score(y_test, y_pred_test),recall_score(y_test, y_pred_test),f1_score(y_test, y_pred_test),roc_auc_score(y_test, y_pred_test)]

df['tabnet_train'] = tab_train
df['tabnet_test'] = tab_test


print(confusion_matrix(y_test,y_pred_test))

[[3346   46]
 [ 162  143]]


In [291]:
df.index = ['accuracy','precision','recall','f1-score','roc-auc']
df

,logit_train,logit_test,rf_train,rf_test,xgb_train,xgb_test,lgb_train,lgb_test,cat_train,cat_test,svm_train,svm_test,tabnet_train,tabnet_test
accuracy,0.944928,0.944279,1.0,0.945632,0.999768,0.945091,0.992696,0.944820,0.985391,0.945902,0.940406,0.941304,0.941217,0.943738
precision,0.772414,0.756477,1.0,0.726087,1.000000,0.694656,0.968254,0.696498,0.986711,0.714286,0.823529,0.814286,0.737819,0.756614
recall,0.471910,0.478689,1.0,0.547541,0.997191,0.596721,0.942416,0.586885,0.834270,0.573770,0.353933,0.373770,0.446629,0.468852
f1-score,0.585876,0.586345,1.0,0.624299,0.998594,0.641975,0.955160,0.637011,0.904110,0.636364,0.495088,0.512360,0.556430,0.578947
roc-auc,0.729700,0.732416,1.0,0.764484,0.998596,0.786568,0.969818,0.781945,0.916629,0.776567,0.673554,0.683053,0.716174,0.727646
